In [ ]:
from keras.applications.vgg16 import VGG16, VGG19
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np

import matplotlib.pyplot as plt

from keras_svm import sequential_with_svm

# Keras imports

from keras.layers import Input, Conv2D, Convolution2D, MaxPooling2D, Activation, Average, concatenate, Dropout, GlobalAveragePooling2D, Flatten, Dense
from keras.models import Model
from keras import regularizers, optimizers
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.preprocessing.image import load_img, img_to_array
from keras.datasets import cifar10
from keras.callbacks import TensorBoard, ModelCheckpoint
try:
    from sklearn.cross_validation import StratifiedShuffleSplit
except:
    from sklearn.model_selection import StratifiedShuffleSplit
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

import csv

In [ ]:
# Fire Module Definition
sq1x1 = "squeeze1x1"
exp1x1 = "expand1x1"
exp3x3 = "expand3x3"
relu = "relu_"

def fire_module(x, fire_id, squeeze=16, expand=64):
    s_id = 'fire' + str(fire_id) + '/'

    channel_axis = 3
    
    x = Convolution2D(squeeze, (1, 1), padding='valid', name=s_id + sq1x1)(x)
    x = Activation('relu', name=s_id + relu + sq1x1)(x)

    left = Convolution2D(expand, (1, 1), padding='valid', name=s_id + exp1x1)(x)
    left = Activation('relu', name=s_id + relu + exp1x1)(left)

    right = Convolution2D(expand, (3, 3), padding='same', name=s_id + exp3x3)(x)
    right = Activation('relu', name=s_id + relu + exp3x3)(right)

    x = concatenate([left, right], axis=channel_axis, name=s_id + 'concat')
    return x

#SqueezeNet model definition
def SqueezeNet(model_input):
#     img_input = Input(shape=input_shape) #placeholder
    img_input = model_input
    
    x = Convolution2D(64, (3, 3), strides=(2, 2), padding='valid', name='squeeze_conv1')(img_input)
    x = Activation('relu', name='relu_conv1')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool1')(x)

    x = fire_module(x, fire_id=2, squeeze=16, expand=64)
    x = fire_module(x, fire_id=3, squeeze=16, expand=64)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool3')(x)

    x = fire_module(x, fire_id=4, squeeze=32, expand=128)
    x = fire_module(x, fire_id=5, squeeze=32, expand=128)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool5')(x)

    x = fire_module(x, fire_id=6, squeeze=48, expand=192)
    x = fire_module(x, fire_id=7, squeeze=48, expand=192)
    x = fire_module(x, fire_id=8, squeeze=64, expand=256)
    x = fire_module(x, fire_id=9, squeeze=64, expand=256)
    
    x = Dropout(0.5, name='drop9')(x)

    x = Convolution2D(1000, (1, 1), padding='valid', name='conv10')(x)
    x = Activation('relu', name='relu_conv10')(x)
    x = GlobalAveragePooling2D()(x)
    x = Activation('softmax', name='loss')(x)

    model = Model(img_input, x, name='squeezenet')

    # Download and load ImageNet weights
    model.load_weights('./squeezenet_weights_tf_dim_ordering_tf_kernels.h5')
    
    return model    

def newSqueezeNet(model_input):
    #Add new classification layers
    x = model_input.layers[-5].output
    x = Convolution2D(7, (1, 1), padding='valid', name='new_conv10')(x)
    x = Activation('relu', name='new_relu_conv10')(x)
    x = GlobalAveragePooling2D()(x)
    x = Activation('softmax', name='new_loss')(x)
    model = Model(squeezeModel.inputs, x, name='squeezenet')
    
    return model

In [ ]:
def conv_pool_cnn(model_input):
    
    x = Conv2D(96, kernel_size=(3, 3), activation='relu', padding =    'same')(model_input)
    x = Conv2D(96, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(96, (3, 3), activation='relu', padding = 'same')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides = 2)(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides = 2)(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (1, 1), activation='relu')(x)
    x = Conv2D(7, (1, 1))(x)
    x = GlobalAveragePooling2D()(x)
    x = Activation(activation='softmax')(x)
    
    model = Model(model_input, x, name='conv_pool_cnn')
    
    return model

In [ ]:
def all_cnn(model_input):
    
    x = Conv2D(96, kernel_size=(3, 3), activation='relu', padding = 'same')(model_input)
    x = Conv2D(96, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(96, (3, 3), activation='relu', padding = 'same', strides = 2)(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same', strides = 2)(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (1, 1), activation='relu')(x)
    x = Conv2D(7, (1, 1))(x)
    x = GlobalAveragePooling2D()(x)
    x = Activation(activation='softmax')(x)
        
    model = Model(model_input, x, name='all_cnn')
    
    return model

In [ ]:
def nin_cnn(model_input):
    
    #mlpconv block 1
    x = Conv2D(32, (5, 5), activation='relu',padding='valid')(model_input)
    x = Conv2D(32, (1, 1), activation='relu')(x)
    x = Conv2D(32, (1, 1), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Dropout(0.5)(x)
    
    #mlpconv block2
    x = Conv2D(64, (3, 3), activation='relu',padding='valid')(x)
    x = Conv2D(64, (1, 1), activation='relu')(x)
    x = Conv2D(64, (1, 1), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Dropout(0.5)(x)
    
    #mlpconv block3
    x = Conv2D(128, (3, 3), activation='relu',padding='valid')(x)
    x = Conv2D(32, (1, 1), activation='relu')(x)
    x = Conv2D(7, (1, 1))(x)
    
    x = GlobalAveragePooling2D()(x)
    x = Activation(activation='softmax')(x)
    
    model = Model(model_input, x, name='nin_cnn')
    
    return model

In [ ]:
def ensemble(models, model_input):
    
    outputs = [model.outputs[0] for model in models]
    y = Average()(outputs)
    
    model = Model(model_input, y, name='ensemble')
    
    return model

In [ ]:
from time import time
import keras.callbacks as callbacks
BATCHSIZE = 40

from keras.utils.np_utils import to_categorical

def compile_and_train(model, num_epochs, train_x, train_labels, val_x, val_labels):
    
    model.compile(loss = "categorical_crossentropy", optimizer = optimizers.Adam(lr=0.0001), metrics=["accuracy"])
    filepath = 'weights/' + model.name + '.{epoch:02d}-{loss:.2f}.hdf5'
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, 
                                 save_weights_only=True, save_best_only=True, 
                                 mode='auto', period=1)
#     tensor_board = TensorBoard(log_dir='logs/', histogram_freq=0, batch_size=32)
#     history = model.fit(x=x_train, y=y_train, batch_size=32, epochs=num_epochs, 
#                         verbose=1, callbacks=[checkpoint, tensor_board], 
#                         validation_split=0.2)
    

    train_categorical_labels = to_categorical(train_labels, num_classes=None)
    val_categorical_labels = to_categorical(val_labels, num_classes=None)

    datagen = ImageDataGenerator(
        featurewise_center=True,
        featurewise_std_normalization=True,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        rescale=1./10,
        fill_mode='nearest',
        horizontal_flip=True,
        vertical_flip=False)

    datagen.fit(train_x)

    #Compile model
    # ...



    tbCallBacks = [callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=7,
                                          verbose=0, mode='auto', baseline=None, 
                                          restore_best_weights=False),
                  TensorBoard(log_dir="./monitor".format(time()), write_graph=True,
                              histogram_freq=0, batch_size=BATCHSIZE),
                  checkpoint]

    #Train model
    # ...

    history = model.fit_generator(datagen.flow(train_x, train_categorical_labels, batch_size=BATCHSIZE),
                        validation_data=datagen.flow(val_x, val_categorical_labels, batch_size=BATCHSIZE),
                        validation_steps=len(val_x)/BATCHSIZE,
                        steps_per_epoch=len(train_x)/BATCHSIZE, 
                        epochs=num_epochs, use_multiprocessing=True,
                        initial_epoch=0,
                        callbacks=tbCallBacks)

In [ ]:
image_np = np.empty((35887, 48, 48, 3))
count = 0
with open('test_data/images.dat', 'r') as csvfile:
    image_csv = csv.reader(csvfile, delimiter=' ')
    for row in image_csv:
        image_np[count] = np.stack((np.reshape(row, (48, 48)),)*3, axis=-1)
        count += 1
with open('test_data/labels.txt', 'r') as labels_file:
    labels = np.asarray(labels_file.readlines()).astype('uint8')

In [ ]:
(x_data, x_label) = image_np, labels

# Prepare the data
# ...
trainVal_x, test_x, trainVal_labels, test_labels = train_test_split(x_data, x_label, test_size=0.2, random_state=0)
train_x, val_x, train_labels, val_labels = train_test_split(trainVal_x, trainVal_labels, test_size=0.2, random_state=0)

print("Train/Val data. X: ", trainVal_x.shape, ", Y: ", trainVal_x.shape)
print("Test data. X: ", test_x.shape, ", Y: ", test_labels.shape)

In [ ]:
from keras.utils.np_utils import to_categorical

train_categorical_labels = to_categorical(train_labels, num_classes=None)
val_categorical_labels = to_categorical(val_labels, num_classes=None)
test_categorical_labels = to_categorical(test_labels, num_classes=None)

In [ ]:
# model = VGG16(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

model = ResNet50(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

In [ ]:
x = model.output
x = Convolution2D(7, (1, 1), padding='valid', name='conv7')(x)
x = Activation('relu', name='relu_7')(x)
x = GlobalAveragePooling2D()(x)
x = Activation('softmax', name='loss')(x)
# x = sequential_with_svm.SequentialWithSvm()(x)

model = Model(model.inputs, x, name='resnet50')

In [ ]:
model.summary()

In [ ]:
modelResNet = compile_and_train(model, 100, train_x, train_labels, val_x, val_labels)

In [ ]:
squeezeModel = SqueezeNet((48, 48, 3))
squeezeModel = newSqueezeNet(squeezeModel)
#new Model

modelSqueeze = compile_and_train(model, 100, train_x, train_labels, val_x, val_labels)

In [ ]:
model = VGG19(weights='imagenet', include_top=False, input_shape=(48, 48, 3))
#Add new classification layers
x = model.output
x = Convolution2D(7, (1, 1), padding='valid', name='conv7')(x)
x = Activation('relu', name='relu_7')(x)
x = GlobalAveragePooling2D()(x)
x = Activation('softmax', name='loss')(x)
# x = sequential_with_svm.SequentialWithSvm()(x)

model = Model(model.inputs, x, name='VGG19')

modelVGG19 = compile_and_train(model, 100, train_x, train_labels, val_x, val_labels)

In [ ]:
input_shape = train_x[0,:,:,:].shape
model_input = Input(shape=input_shape)

In [ ]:
train_x[0,:,:,:].shape

In [ ]:
conv_pool_cnn_model = conv_pool_cnn(model_input)
model_pool_cnn = compile_and_train(conv_pool_cnn_model, 100, train_x, train_labels, val_x, val_labels)

In [ ]:
# squeezeModel = SqueezeNet((48,48,3))
squeezeModel = SqueezeNet(model_input)
squeezeModel = newSqueezeNet(squeezeModel)
squeezeModel.load_weights('./weights/squeezenet.29-0.91.hdf5')

modelResNet = ResNet50(weights='imagenet', include_top=False, input_tensor=model_input)

x = modelResNet.output
x = Convolution2D(7, (1, 1), padding='valid', name='conv7')(x)
x = Activation('relu', name='relu_7')(x)
x = GlobalAveragePooling2D()(x)
x = Activation('softmax', name='loss')(x)
# x = sequential_with_svm.SequentialWithSvm()(x)

modelResNet = Model(modelResNet.inputs, x, name='resnet50')
modelResNet.load_weights('./weights/resnet50.19-0.44.hdf5')

In [ ]:
conv_pool_cnn_model = conv_pool_cnn(model_input)
conv_pool_cnn_model.load_weights('./weights/conv_pool_cnn.18-0.56.hdf5')

In [ ]:
models = [squeezeModel, modelResNet, conv_pool_cnn_model]
ensemble_model = ensemble(models, model_input)

In [ ]:
predict = ensemble_model.predict(test_x, batch_size=40)
predict = np.argmax(predict, axis=1)
predict = np.expand_dims(predict, axis=1)

### (0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral)

In [ ]:
def newSqueezeNet(model_input):
    #Add new classification layers
    x = model_input.layers[-5].output
    x = Convolution2D(7, (1, 1), padding='valid', name='new_conv10')(x)
    x = Activation('relu', name='new_relu_conv10')(x)
    x = GlobalAveragePooling2D()(x)
    x = Activation('softmax', name='new_loss')(x)
    model = Model(squeezeModel.inputs, x, name='squeezenet')
    
    return model

In [ ]:
def evaluate_error(model, x, labels):
    pred = model.predict(x, batch_size = 40)
    pred = np.argmax(pred, axis=1)
    pred = np.expand_dims(pred, axis=1) # make same shape as y_test
    error = np.sum(np.not_equal(pred, labels.reshape((x.shape[0], 1)))) / test_labels.shape[0]
  
    return error

In [ ]:
evaluate_error(ensemble_model, val_x, val_labels)

In [ ]:
test_labels[0]

In [ ]:
expressions = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
count = 0
for i in predict[:100]:
    plt.imshow(test_x[count,:,:,0], cmap='gray')
    plt.show()
    print('predicted: {}, correct: {}'.format(expressions[i[-1]], expressions[test_labels[count]]))
    count += 1